In [51]:
#Parsing an HTML content
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import sys

def progressBar(count_value, total, suffix=''):
    bar_length = 100
    filled_up_Length = int(round(bar_length* count_value / float(total)))
    percentage = round(100.0 * count_value/float(total),1)
    bar = '=' * filled_up_Length + '-' * (bar_length - filled_up_Length)
    sys.stdout.write('[%s] %s%s ...%s\r' %(bar, percentage, '%', suffix))
    sys.stdout.flush()

In [ ]:
#check the soup!
URL = "https://sinta.kemdikbud.go.id/affiliations/authors/430?page=1"
r = requests.get(URL)
  
soup = BeautifulSoup(r.content, 'html5lib')
print(soup.prettify())

In [52]:
#start scraping
nama=[];sintascore=[];ID=[];departemen=[];sinta3=[];sinta=[];affilsc3=[];affilsc=[];scopus=[];GSH=[]
for i in range(0,290):  #290 pages
    progressBar(i+1, 290)
    url="https://sinta.kemdikbud.go.id/affiliations/authors/430?page="
    path=url+str(i+1)
    r = requests.get(path)
    
    soup = BeautifulSoup(r.content, 'html5lib')
    
    table = soup.find('div', attrs = {'class':'content'})                                       #get body

    for i in table.findAll('div', attrs = {'class':'au-item mt-3 mb-3 pb-5 pt-3'}):             #get specific div
        nama.append(i.find("div", attrs = {"class": "profile-name"}).text)
        departemen.append(i.find("div", attrs={"class" : "profile-dept"}).text.strip(' \n\t'))
        ID.append(i.find("div", attrs={"class" : "profile-id"}).text)
        scopus.append(i.find("span", attrs={"class" : "profile-id text-warning"}).text)
        GSH.append(i.find("span", attrs={"class" : "profile-id text-success ml-3"}).text)
        
        j=i.findAll("div", attrs={"class" : "stat-num text-center"})
        sinta3.append(j[0].text)
        sinta.append(j[1].text)
        affilsc3.append(j[2].text)
        affilsc.append(j[3].text)
    
#get numeric value of data from string
ID=[item for subitem in ID for item in subitem.split() if item.isdigit()]
scopus=[item for subitem in scopus for item in subitem.split() if item.isdigit()]
GSH=[item for subitem in GSH for item in subitem.split() if item.isdigit()]

In [ ]:
print(len(ID))              #amount of lecturers

In [55]:
pd.DataFrame.to_csv(pd.DataFrame({"nama": nama,
              "departemen" : departemen,
              "ID": ID,
              "Scopus H-Index": scopus,
              "GS H-Index" : GSH,
              "SINTA_score_3yr": sinta3,
              "SINTA_score": sinta,
              "Affil_score_3yr": affilsc3,
              "Affil_score" : affilsc
              }), "Lol.csv")